In [1]:
from jupyter_utils import jupyter_setup
jupyter_setup()
import numpy as np
import pandas as pd
import torch
from torch import nn
from torchvision.models import resnet18, resnet50, ResNet18_Weights, ResNet50_Weights


D:\Archiv\Studium\Master\6.-Semester\Masters_Thesis\Git\acoustic_covid_detection\python


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def get_parameter_groups(model, output_lr, input_lr, verbose=True):
    # applies different learning rates for each (parent) layer in the model (for finetuning a pretrained network).
    # the inout layer gets the input_lr, the output layer the output_lr. All layers in between get linearly interpolated.

    # works for resnet architecture and assigns a learning rate for each parent layer and the input and output layers
    # in total there are (for a resnet 18) 61 parameter groups but only 4 parent layers and 3 layers as input/output layers
    # this means there are only  4+3  different learning rates.
    
    parent_layer = lambda name: name.split(".")[0]    
    layer_names = [name for name, _ in model.named_parameters()]
    layer_names.reverse()
    parent_layers = list(set([parent_layer(layer) for layer in layer_names]))
    n_parent_layers = len(parent_layers)
    lr=output_lr
    last_parent_layer = parent_layer(layer_names[0])
    if verbose:
        print(f'0: lr = {lr:.6f}, {last_parent_layer}')
    
    lr_mult = np.power(input_lr/output_lr, 1/(n_parent_layers-1))
    parameters = []
    for idx, layer in enumerate(layer_names):
        current_parent_layer = parent_layer(layer)
        if last_parent_layer != (current_parent_layer):
            lr *= lr_mult
            if verbose:
                print(f'{idx}: lr = {lr:.6f}, {current_parent_layer}')
            last_parent_layer = current_parent_layer
        parameters.append({'params': [p for n, p in model.named_parameters() if n == layer and p.requires_grad],
                           'lr':     lr})
    return parameters

In [ ]:
my_model = resnet18(weights=ResNet18_Weights.DEFAULT)

In [ ]:
parameters = get_parameter_groups(my_model, output_lr=1e-3, input_lr=5e-5, verbose=True)

In [ ]:
my_model

In [26]:
a = torch.arange(30).view((5,2,3))
b = (torch.arange(30)+100).view((5,2,3))
c = (torch.arange(30)+1000).view((5,2,3))

# a = a.unsqueeze(0)
# b = b.unsqueeze(0)
# c = c.unsqueeze(0)
a.shape

torch.Size([5, 2, 3])

In [27]:
abc = torch.stack([a, b, c])
abc.shape

torch.Size([3, 5, 2, 3])

In [33]:
abc[0]

tensor([[[ 0,  1,  2],
         [ 3,  4,  5]],

        [[ 6,  7,  8],
         [ 9, 10, 11]],

        [[12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23]],

        [[24, 25, 26],
         [27, 28, 29]]])

In [36]:
abc_reshaped = abc.view(15,2,3)
abc_reshaped.shape

torch.Size([15, 2, 3])

In [39]:
abc_reshaped[3]

tensor([[18, 19, 20],
        [21, 22, 23]])